# CapiPort - PORTFOLIO OPTIMISATION

    Two things to consider for Portfolio Optimisation:
    
        1) Minimising Risk
        2) Maximising Return

    Basic process of Portfolio Optimisation:
        
        1) Select the Asset class to work on.
            1.1) Asset Class choosen - Equity (Stocks)
        2) Select the Companies which you want to use to build a Portfolio.
            2.1) Companies choosen - 
        3) To try various Statistical Methods relating to Portfolio Optimisation.
            3.1) Method 1 - Result
            3.2) Method 2 - Result
        4) You will obtain Weigths or Percentages of Portfolio to invest.
            4.1) Method 1 - Weights
            4.2) Method 2 - Weights
        5) Testing the Portfolio for the future.
            5.1) Method 1 - Result
            5.2) Method 2 - Result
        6) Final Result

## <u>STEPS FOR IMPLEMENTING<u>

    1) IMPORTING THE LIBRARIES
    2) TWEETS EXTRACTION FROM STOCKNET
    3) TWITTER DATA PRE-PROCESSING
    4) ZERO-SHOT SENTIMENT CLASSIFICATION
    5) FEATURE ENGINEERING OF TWEETS SENTIMENT VALUES
        5.1) Number of Tweets for each individual days
        5.2) Average of Emotion for each individual days
        5.3) Median of Sentiment for each Single Day
        5.4) Maximum Sentiment Value for each Single day
        5.5) Minimum Sentiment Value for Each Single Day
        5.6) Combining all the dataframes
    6) STOCK DATA FROM STOCKNET
    7) STOCK DATA AND FEATURE ENGINEERED SENTIMENT VALUES MERGING STEP
    9) WITH SENTIMENT
        9.1) DATASET PREPARATION FOR TRAINING
        9.2) TRAINING
        9.3) EVALUATING
        9.4) GRAPHS AND METRICS